## Задание 08
В задании не будет подробных инструкций. Ваша задача - построить классификатор с как можно лучшим качеством (AUC-ROC). 

In [80]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from xgboost import XGBClassifier
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import GridSearchCV
from catboost import CatBoostClassifier, Pool
from sklearn.preprocessing import OneHotEncoder

Будем работать с данными по авиарейсам в США. Задача  -  предсказать задержку вылета более 15 минут (задача бинарной классификации).

Признаки:

* Month, DayofMonth, DayOfWeek, месяц, день месяца и день недели
* DepTime, время отправления
* UniqueCarrier, код перевозчика
* Origin, место вылета
* Dest, место назначения
* Distance, расстояние между аэропортами вылета и прилета
* dep_delayed_15min, просрочка вылета на 15 и более минут (целевой признак)


In [81]:
df = pd.read_csv('flight_delays.csv')

In [82]:
df.tail()

,Month,DayofMonth,DayOfWeek,DepTime,UniqueCarrier,Origin,Dest,Distance,dep_delayed_15min
99995,c-5,c-4,c-3,1618,OO,SFO,RDD,199,N
99996,c-1,c-18,c-3,804,CO,EWR,DAB,884,N
99997,c-1,c-24,c-2,1901,NW,DTW,IAH,1076,N
99998,c-4,c-27,c-4,1515,MQ,DFW,GGG,140,N
99999,c-11,c-17,c-4,1800,WN,SEA,SMF,605,N


В качестве простейшего бенчмарка возьмем логистическую регрессию и два признака, которые проще всего взять: `DepTime` и `Distance`:

In [83]:
X, y = df[['Distance', 'DepTime']].values, df['dep_delayed_15min'].map({'Y': 1, 'N': 0}).values
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.3,random_state=17)

In [84]:
lr = LogisticRegression( solver='lbfgs',random_state=17)

lr.fit(X_train, y_train)
y_pred = lr.predict_proba(X_test)[:, 1]

print('AUC-ROC:',roc_auc_score(y_test, y_pred))

AUC-ROC: 0.6795697123357751


####  Критерии оценивания:

#### 1. Предобработка данных  - 3 балла 

Предобработка включает one-hot кодирование категориальных признаков, заполнение пропусков (если есть),  генерацию новых признаков (например, вместо Origin и Dest можно ввести признак маршрут: Origin-Dest)

#### 2. Настройка параметров обучения - (4 балла) 
Настройка включает определение гиперпараметров бустинга (предполагается, что вы будете использовать XGBoost) на кросс-валидации: глубина деревьев, темп обучения, число деревьев и т.д. 

#### 3. Использование стекинга, отбора признаков и другое - (3 балла)

Например, можно реализовать простейшую схему стекинга - блендинг:
* обучите логистическую регрессию и градиентный бустинг;
* постройте линейную смесь ответов логистической регрессии и градиентного бустинга вида 
$$p=w_1∗p_{lr}+(1−w_1)∗p_{xgb},$$ 
где  $p_{lr}$  – предсказанные логистической регрессией вероятности класса 1,  $p_{xgb}$ – бустинга. 

In [91]:
# df.isna().sum() - Nan is none
# New features: Month-Day-Origin and Month-Day-Destination, encoded as mean tareget with regularization
data = df.copy()
data['Origin-Dest'] = data['Origin'] + '-' + data['Dest']
data['dep_delayed_15min'] = data['dep_delayed_15min'].map({'Y': 1, 'N': 0}).values
y = data['dep_delayed_15min']
data = data.drop(columns=['dep_delayed_15min'])
data = pd.get_dummies(data)
X_train, X_test, y_train, y_test = train_test_split(data, y, test_size=0.3)

In [ ]:
eval_set = [(X_test, y_test)]
xgb = XGBClassifier(max_depth=5, n_estimators=100)
params0 = {'max_depth':[2]}
params = {'max_depth': [2,4,6,8, 10], 'n_estimators':[50, 100, 150, 200], 'learning_rate':np.linspace(0.0, 1.0, num=10)}
#xgb_grid = GridSearchCV(xgb, params0, verbose=True, scoring = 'roc_auc', n_jobs = 2, cv = 2)
xgb.fit(X_train, y_train, eval_metric="auc", eval_set=eval_set, verbose=True, early_stopping_rounds=10)
y_pred = xgb.predict_proba(X_test)[:,1]
print('AUC-ROC:',roc_auc_score(y_test, y_pred))
#print(xgb.best_score_)
#print(xgb.best_params_)

In [ ]:
# Пробовал mean-target кодирование для признаков Date-Origin и Date-Dest,
#      качество сильно упало, код оставил на всякий случай, на будущее 
# New features: Month-Day-Origin and Month-Day-Destination, encoded as mean tareget with regularization
data = df.copy()
data['DateOrigin'] = data['Month'] + data['DayofMonth'] + data['Origin']
data['DateDest'] = data['Month'] + data['DayofMonth'] + data['Dest']
data['dep_delayed_15min'] = data['dep_delayed_15min'].map({'Y': 1, 'N': 0}).values
y = data['dep_delayed_15min']
data = data.drop(columns=['Origin'])

data_train, data_test, y_train, y_test = train_test_split(data, y, test_size=0.3)

# for encoding use only train samples in order to prevent knowledge leaking
C = 1
y_rate = data_train['dep_delayed_15min'].sum() / len(data_train)

for encoding_feature in ['DateOrigin', 'DateDest']:
    counts = data_train.groupby(encoding_feature).size()
    y_counts = data_train.groupby(encoding_feature)['dep_delayed_15min'].value_counts()[:,1]

    encoding_df_temp = pd.DataFrame(counts).join(pd.DataFrame(y_counts), on=encoding_feature, how='left').fillna(0)
    encoding_df_temp.columns = ['Count', '1_count']
    encoding_df_temp = pd.DataFrame((encoding_df_temp['1_count'] + C * y_rate)/ (encoding_df_temp['Count'] + C))
    encoding_df_temp.columns = [encoding_feature + 'Mean']
    
    data_train = data_train.join(encoding_df_temp, on=encoding_feature, how='left')
    data_test = data_test.join(encoding_df_temp, on=encoding_feature, how='left')
    data_test = data_test.fillna(0)

print(data_train.head())
X_train = data_train.drop(columns=['dep_delayed_15min', 'DateOrigin', 'DateDest', 'DayofMonth', 'Month'])
X_test = data_test.drop(columns=['dep_delayed_15min', 'DateOrigin', 'DateDest', 'DayofMonth', 'Month'])
X_train = pd.get_dummies(X_train)
X_test = pd.get_dummies(X_test)